In [17]:
from pathlib import Path
import hashlib
import google.generativeai as genai
from dotenv import load_dotenv
import os

In [18]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [19]:

GAME_THEME = "technology" # Change this later to auto fetch based on GAME_KEY
GAME_LANGUAGE = "Arabic"

In [20]:
genai.configure(api_key=GEMINI_API_KEY)

# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
  "response_mime_type": "application/json",
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)


In [21]:
uploaded_files = []
def upload_if_needed(pathname: str) -> list[str]:
  path = Path(pathname)
  hash_id = hashlib.sha256(path.read_bytes()).hexdigest()
  try:
    existing_file = genai.get_file(name=hash_id)
    return [existing_file]
  except:
    pass
  uploaded_files.append(genai.upload_file(path=path, display_name=hash_id))
  return [uploaded_files[-1]]

In [22]:
path_of_image = '../assets/mouse_keyboard.jpg'

prompt_parts = [
  "input:Tell me all the objects you see in this image and their confidence values and pixel coordinates, only if the confidence value is greater than 0.7 and if the object that fits best with the category: {GAME_THEME}.",
  *upload_if_needed(path_of_image),
  "output: ",
]

response = model.generate_content(prompt_parts)

In [23]:
import json
json_response = json.loads(response.parts[0].text)

In [24]:
json_response

[{'object': 'computer keyboard',
  'confidence': 0.99,
  'x_min': 126,
  'x_max': 878,
  'y_min': 281,
  'y_max': 550},
 {'object': 'computer mouse',
  'confidence': 0.99,
  'x_min': 772,
  'y_min': 605,
  'x_max': 957,
  'y_max': 760}]

In [25]:
number_of_objects = len(json_response)

import random
picked_object = random.choice(json_response)
picked_object

{'object': 'computer keyboard',
 'confidence': 0.99,
 'x_min': 126,
 'x_max': 878,
 'y_min': 281,
 'y_max': 550}

In [26]:
picked_object_word = picked_object['object']
picked_object_x_coordinate = (picked_object['x_max'] + picked_object['x_min']) / 2
picked_object_y_coordinate = (picked_object['y_max'] + picked_object['y_min']) / 2

picked_object_word, picked_object_x_coordinate, picked_object_y_coordinate

('computer keyboard', 502.0, 415.5)

In [27]:
# Set up the model
image_checker_generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 8192,
    "response_mime_type": "application/json",
}

image_checker_model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                                            generation_config=image_checker_generation_config,
                                            safety_settings=safety_settings)

In [32]:
checker_prompt_parts = [
  "input: Tell me whether or not the word {picked_object_word} is related to the category of {GAME_THEME}. Limit your output to 'True' or 'False'.",
  "output: ",
]

In [33]:
checker_response = image_checker_model.generate_content(checker_prompt_parts)

In [34]:
checker_response_json_response = json.loads(checker_response.parts[0].text)

In [36]:
checker_response_json_response

['False']

In [35]:
related_word_generator_prompt_parts = [
  "input: Give me 5 words related to the word {picked_object_word} and fall under the {GAME_THEME} category.",
  "output: ",
]

In [16]:
related_word_generator_response = model.generate_content(related_word_generator_prompt_parts)

In [17]:
related_word_generator_json_response = json.loads(related_word_generator_response.parts[0].text)

In [18]:
related_word_generator_json_response

['pumpkin', 'costume', 'haunted', 'candy', 'spooky']

In [1]:
from googletrans import Translator
translator = Translator()

In [26]:
translated_picked_object_word = translator.translate(picked_object_word, dest=GAME_LANGUAGE).text
translated_related_words = [translator.translate(word, dest=GAME_LANGUAGE).text for word in related_word_generator_json_response]

translated_related_word1 = translated_related_words[0]
translated_related_word2 = translated_related_words[1]
translated_related_word3 = translated_related_words[2]
translated_related_word4 = translated_related_words[3]
translated_related_word5 = translated_related_words[4]

In [28]:
translated_related_word1

'يقطين'